In [107]:
import numpy as np
import pandas as pd
import scipy.stats 
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

import time, os, pickle, joblib, datetime, random, warnings
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor as rf
from sklearn.linear_model import LinearRegression as LR
from sklearn.model_selection import train_test_split as tts
from sklearn.model_selection import cross_val_score as cvs
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import StandardScaler,scale
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.utils import shuffle
from sklearn.ensemble import ExtraTreesRegressor
from xgboost import XGBRegressor as XGBR
from sklearn import metrics
from sklearn.preprocessing import RobustScaler
from xgboost import plot_tree, plot_importance
from sklearn.model_selection import KFold
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  #解决新版本GPU报错的问题
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping
import seaborn as sns
from scipy.stats import gaussian_kde
import joblib, shap

tf.__version__

'2.10.0'

In [ ]:
import ee, geemap, os
import geopandas as gpd

ee.Authenticate()
ee.Initialize()

os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890' 
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'

Map = geemap.Map(center=(40, -100), zoom=1,height = 500,width = 800)
Map

## MK trend analysis

In [ ]:
import pandas as pd
import pymannkendall as mk
df = pd.read_excel()

trend_list = []
for i in range(len(df)):
    data = df.iloc[i,4:-6].tolist()
    data = list(filter(lambda x: isinstance(x, (int, float)) and not np.isnan(x), data))
    if len(data) <= 5:
        print(0)
        continue
    try:
        res = mk.original_test(data,alpha = 0.1)
        print(res.trend,round(res.p,2),round(res.slope,6),sep = ',')
        if res.trend == 'no trend':
            trend_list.append(0)
            # print(0)
        if res.trend == 'increasing':
            trend_list.append(1)
            # print(1)
        if res.trend == 'decreasing':
            trend_list.append(-1)
            # print(-1)
    except:
        # print('none')
        trend_list.append(0)

# Deforestation analysis

In [24]:
dataset = ee.Image('UMD/hansen/global_forest_change_2023_v1_11')

first_Bands = ['first_b30', 'first_b40', 'first_b50',]
last_Bands = ['last_b30', 'last_b40', 'last_b50']

first_image = dataset.select(first_Bands)
last_image  = dataset.select(last_Bands)

Map.addLayer(first_image, {'bands':first_Bands,'gamma':1.5}, 'Year 2000 Bands 5/4/3')
Map.addLayer(last_image, {'bands':last_Bands,'gamma':1.5}, 'Year 2022 Bands 5/4/3')

In [39]:
basin = basin.to_crs(epsg=5070)

basin['area_m2'] = basin.geometry.area

basin['area_km2'] = basin['area_m2'] / 1_000_000

basin['area_m2'] = basin['area_m2'].apply(lambda x: f"{x:,.0f}")
basin['area_km2'] = basin['area_km2'].apply(lambda x: f"{x:,.0f}")

print(basin[['area_m2', 'area_km2']])

             area_m2   area_km2
0  3,228,837,603,285  3,228,838


In [ ]:
basins = ee.FeatureCollection('projects/sat-io/open-datasets/HydroAtlas/BasinAtlas/BasinATLAS_v10_lev04')

for river_num in range(297, len(river)+1):  # len(river) + 1
    selected_point = river[river['rivernum'] == river_num]
    geometry = geemap.geopandas_to_ee(selected_point)
    Map.addLayer(geometry, {}, 'Geometry')

    basin_shp = basins.filterBounds(geometry)

    Map.addLayer(basin_shp, {'color': 'blue'}, 'Basin at Point')
    Map.centerObject(basin_shp, 6)

    for threshold in [20]:
        dataset = ee.Image('UMD/hansen/global_forest_change_2023_v1_11')
        tree_cover = dataset.select('treecover2000').gte(threshold).selfMask()
        masked_tree_cover = tree_cover.clip(basin_shp)

        forest_cover_area = masked_tree_cover.multiply(ee.Image.pixelArea()).reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=basin_shp.geometry(),
            scale=30, 
            maxPixels=1e13
        )

        forest_area_km2 = forest_cover_area.get('treecover2000').getInfo() / 1_000_000

        loss = dataset.select('loss')
        loss = loss.updateMask(loss).clip(basin_shp)

        loss_area = loss.multiply(ee.Image.pixelArea()).reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=basin_shp.geometry(),
            scale=30, 
            maxPixels=1e13
        )

        loss_area_km2 = loss_area.get('loss').getInfo() / 1_000_000

        gain = dataset.select('gain')
        gain = gain.updateMask(gain).clip(basin_shp)

        gain_area = gain.multiply(ee.Image.pixelArea()).reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=basin_shp.geometry(),
            scale=30,
            maxPixels=1e13
        )

        gain_area_km2 = gain_area.get('gain').getInfo() / 1_000_000

        try:
            deforestation = (loss_area_km2 - gain_area_km2) / forest_area_km2

            print('河段: {} | 阈值: {} | 森林覆盖面积（平方千米）: {} | 损失面积: {} | 增益面积: {} | 森林损失率: {:.2f}%'.format(
                river_num,
                threshold,
                int(forest_area_km2),
                int(loss_area_km2),
                int(gain_area_km2),
                round(deforestation * 100, 2)
            ))
        except:
            print('河段',river_num)

# Dam analysis

In [ ]:
basins = ee.FeatureCollection('projects/sat-io/open-datasets/HydroAtlas/BasinAtlas/BasinATLAS_v10_lev03')

for river_num in range(1,len(river) + 1):  
    selected_point = river[river['rivernum'] == river_num]
    geometry = geemap.geopandas_to_ee(selected_point)
    Map.addLayer(geometry, {}, 'Geometry')

    basin_shp = basins.filterBounds(geometry)

    Map.addLayer(basin_shp, {'color': 'blue'}, 'Basin at Point')
    Map.centerObject(basin_shp, 6)

    basin_shp_ee = basin_shp
    
    basin_shp_data = basin_shp_ee.getInfo()
    features = basin_shp_data['features']
    
    basin_shp_gdf = gpd.GeoDataFrame.from_features(features)

    try:
        intersecting_dams = gpd.sjoin(dams, basin_shp_gdf, how='inner', op='intersects')
        
        num_dams_in_basin = intersecting_dams.shape[0]
        
        reservior_capacity = round(sum(intersecting_dams['CAP_MCM'].tolist())*0.001,2)
        capacity_vs_discharge = round(reservior_capacity/df['ave_discharge'][river_num+1],4)*100
        
        print(river_num, num_dams_in_basin, reservior_capacity, capacity_vs_discharge)
        
    except:
        print(river_num)
        continue

# Land greening analysis

In [ ]:
from sklearn.linear_model import LinearRegression

basins = ee.FeatureCollection('projects/sat-io/open-datasets/HydroAtlas/BasinAtlas/BasinATLAS_v10_lev03')

for river_num in range(len(river)): 
    selected_point = river[river['rivernum'] == river_num]
    geometry = geemap.geopandas_to_ee(selected_point)
    Map.addLayer(geometry, {}, 'Geometry')

    basin_shp = basins.filterBounds(geometry)

    Map.addLayer(basin_shp, {'color': 'blue'}, 'Basin at Point')
    Map.centerObject(basin_shp, 6)
    
    results = []
    for year in range(1984, 2021):
        ndvi = (ee.ImageCollection('NOAA/CDR/AVHRR/NDVI/V5')
                 .filterDate(f'{year}-01-01', f'{year}-12-31')
                 .mean()
                 .select('NDVI')
                 .clip(basin_shp))
        try:
            mean_value = ndvi.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=basin_shp,
                scale=10000,
                maxPixels=1e13
            ).get('NDVI').getInfo()
            
        except:
            mean_value = 0
        year_mean = {'year': year, 'mean': mean_value}
        results.append(year_mean)
        
    years = np.array([result['year'] for result in results]).reshape(-1, 1)
    means = np.array([result['mean'] for result in results]).reshape(-1, 1)

    model = LinearRegression()
    try:
        model.fit(years, means)
    except:
        print('river_num: ',river_num)
        continue
    slope = model.coef_[0][0]
    intercept = model.intercept_[0]
    
    print(f"river_num: {river_num}, Slope: {slope/means.mean():.4f}")

# Warmming analysis

In [ ]:
from sklearn.linear_model import LinearRegression

for river_num in numbers:
    selected_point = river[river['rivernum'] == river_num]
    geometry = geemap.geopandas_to_ee(selected_point)
    Map.addLayer(geometry, {}, 'Geometry')

    basin_shp = basins.filterBounds(geometry)

    Map.addLayer(basin_shp, {'color': 'blue'}, 'Basin at Point')
    Map.centerObject(basin_shp, 4)
    
    results = []
    
    for year in range(1984, 2021):
        
        try:
            temp =  ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY_AGGR') \
                .filterDate(f'{year}-01-01', f'{year}-12-31') \
                .mean().clip(basin_shp) \
                .reduceRegion(reducer=ee.Reducer.mean(),geometry=basin_shp,scale=5000,maxPixels = 1e13).getInfo().get('temperature_2m')
        except:
            temp = 0

        try:
            prec =  ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY_AGGR') \
                .filterDate(f'{year}-01-01', f'{year}-12-31') \
                .sum() \
                .clip(basin_shp) \
                .reduceRegion(reducer=ee.Reducer.mean(),geometry=basin_shp,scale=5000,maxPixels = 1e13).getInfo().get('total_precipitation_sum')
        except:
            prec = 0
            
        year_data = {
        'year': year,'temp': temp,'prec': prec,}
        
        results.append(year_data)

    df = pd.DataFrame(results)
    slope_list = []
    
    variables = ['temp', 'prec']
    for var in variables:
        y = df[var][df[var] != 0].values
        X = np.arange(1, len(y) + 1).reshape(-1, 1)  # 转换为列向量（sklearn 要求 2D 数据）
    
        model = LinearRegression()
        model.fit(X, y)
        
        slope = model.coef_[0]
        mean_val = y.mean()
        change_rate =  round((slope / mean_val) * 100, 4) if mean_val != 0 else np.nan
        slope_list.append(change_rate)
        
    print(river_num, *slope_list, sep='\t')

# Figure mapping

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.pyplot import MultipleLocator,FormatStrFormatter
from pylab import mpl
from sklearn.metrics import mean_squared_error 
from matplotlib import colors, cm
import seaborn as sns
from sklearn.linear_model import LinearRegression
from scipy.stats import t
from scipy.interpolate import make_interp_spline
from matplotlib.colors import ListedColormap,Normalize
from scipy import stats
import mpl_scatter_density
plt.rcParams["font.family"] = 'Arial'

## Fig.1 mapping

In [ ]:
mpl.rcParams['font.size'] = 12

robinson_proj4 = '+proj=robin +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs'
world = world.to_crs(crs=robinson_proj4)
poc = poc.to_crs(crs=robinson_proj4)
country = country.to_crs(crs=robinson_proj4)

fig, ax = plt.subplots(1, 1, figsize=(7, 7.5), dpi=300)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.axis('off')

world.plot(ax=ax, color='#EDEDED', edgecolor=None, linewidth=0.5, alpha = 0.6)

sizes = [20, 60, 200]
poc['size'] = [sizes[0] if 0 <= x < 100 
               else sizes[1] if 100 <= x < 2000 
               else sizes[2] for x in poc['discharge']]

color_list = ['#a50026','#d73027','#f46d43','#fdae61','#fee090','#e0f3f8','#abd9e9','#74add1','#4575b4','#313695']
color_list.reverse()

poc['color'] = [color_list[0] if (x < 0.331600)
                else color_list[1] if (0.331600 <= x < 0.890556)
                else color_list[2] if (0.890556 <= x < 1.226190)
                else color_list[3] if (1.226190 <= x < 1.591940)
                else color_list[4] if (1.591940 <= x < 2.003750)
                else color_list[5] if (2.003750 <= x < 2.484440)
                else color_list[6] if (2.484440 <= x < 3.109390)
                else color_list[7] if (3.109390 <= x < 4.241670)
                else color_list[8] if (4.241670 <= x < 5.883570)
                else color_list[9] if (5.883570 <= x < 8.999120)
                else None for x in poc['poc_ave']]

for x, y, color, size in zip(poc.geometry.x, poc.geometry.y, poc['color'],  poc['size']):
    ax.scatter(x, y,  s=size, edgecolor='k', c=color, zorder = 100, lw = 0.4, alpha = 0.9)

labels = [1, 20, 60]
legend_elements = [plt.scatter([], [], marker='o',  facecolor='none',edgecolor = 'k', lw = 0.5,
                               s=size, label=label) for size, label in zip(sizes, labels)]
legend = plt.legend(handles=legend_elements,markerfirst=True, frameon=False, bbox_to_anchor=(0.68, 0.44),fontsize=7,labelspacing = 1.)  
legend.set_title('Avg. Discharge\n(100 km³/yr)', prop={'size': 7})

cmap = mcolors.ListedColormap(color_list)
bounds = [0, 0.3, 0.9, 1.2, 1.6, 2, 2.5, 3.1, 4.2, 6, 9]
norm = mcolors.BoundaryNorm(bounds, cmap.N)

cax = fig.add_axes([0.82, 0.48, 0.01, 0.12])
colorbar = plt.colorbar(plt.cm.ScalarMappable(cmap=cmap, norm=norm), cax=cax)
colorbar.ax.set_title('POC (mg/L)', fontsize=8, x=2, y=1.05)
colorbar.ax.tick_params(labelsize=8)

plt.show()

## Fig.2 mapping

In [ ]:
plt.rcParams.update({'axes.edgecolor': color, 'axes.labelcolor': color, 'xtick.color': color, 'ytick.color': color})

poc = poc.sort_values(by='discharge', ascending=False)

robinson_proj4 = '+proj=robin +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs'

world = world.to_crs(crs=robinson_proj4)
poc = poc.to_crs(crs=robinson_proj4)

fig, ax = plt.subplots(1, 1, figsize=(8, 7.5), dpi=300)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.axis('off')

world.plot(ax=ax, color='#EDEDED', edgecolor=None, linewidth=0.5, alpha = 0.6)

sizes = [30, 50, 70, 120]
boundary = [0, 0.1, 0.5, 1, 15]

poc['size'] = [sizes[0] if boundary[0] < x < boundary[1]
               else sizes[1] if boundary[1] <= x < boundary[2]
               else sizes[2] if boundary[2] <= x < boundary[3]
               else sizes[3] for x in poc['poc_flux']]

color_list_decrease = ['#023858','#045a8d','#ece7f2','#fff7fb']
color_list_increase = ['#7f0000','#b30000','#fee8c8','#fff7ec']

bounds = [-1.5, -0.05, -0.02, 0, 0.05, 0.1, 0.3, 0.7]

poc['color'] = [
                 '#2166ac' if (x*10 < bounds[1]) & (y == 'decreasing')
                else '#4393c3' if (bounds[1] <= x*10 < bounds[2]) & (y == 'decreasing')
                else '#92c5de' if (bounds[2] <= x*10 < bounds[3]) & (y == 'decreasing')
                else '#DEDEDE' if y == 'no trend'
                else '#ef6548'  if (bounds[3] < x*10 < bounds[4]) & (y == 'increasing')
                else '#d7301f' if (bounds[4] <= x*10 < bounds[5]) & (y == 'increasing')
                else '#b30000' if (bounds[5] <= x*10 < bounds[6]) & (y == 'increasing')
                else '#7f0000' for x,y in zip(poc['slope'], poc['trend'])]

def custom_symbol(trend):
    if trend == 'increasing':
        return '^'
    elif trend == 'decreasing':
        return 'v'
    else:
        return ''

poc['symbol'] = poc['trend'].apply(custom_symbol)

for x, y, color, symbol, size in zip(poc.geometry.x, poc.geometry.y, poc['color'], poc['symbol'], poc['size']):
    ax.scatter(x, y, marker=symbol, s=size, edgecolor='k', c=color, zorder = 100, lw = 0.5, alpha = 0.9)

labels = [f"{boundary[i]} - {boundary[i + 1]}" for i in range(len(boundary) - 1)] + [">1"]
legend_elements = [plt.scatter([], [], marker='^',  facecolor='none', edgecolor = 'k',lw = 0.5,
                               s=size, label=label) for size, label in zip(sizes, labels)]
legend = plt.legend(handles=legend_elements,markerfirst=True, frameon=False, bbox_to_anchor=(0.67, 0.46),fontsize=8,labelspacing = 0.8)  
# legend.set_title('Avg. Discharge\n(100 km³/yr)', prop={'size': 8})
legend.set_title('Avg. POC flux\n(Mt/yr)', prop={'size': 8})
legend.get_title().set_ha("center")
legend._legend_box.align = "center"

color_list =  ['#2166ac', '#4393c3', '#92c5de', '#ef6548', '#d7301f', '#b30000', '#7f0000']
cmap = mcolors.ListedColormap(color_list)
bounds = bounds
norm = mcolors.BoundaryNorm(bounds, cmap.N)

cax = fig.add_axes([0.21, 0.33, 0.008, 0.12])
colorbar = plt.colorbar(plt.cm.ScalarMappable(cmap=cmap, norm=norm), cax=cax)
colorbar.ax.set_title('POC flux change\n rate (Mt/10 yr)', fontsize=8, x=3., y=1.02, linespacing = 1.2)  # x,y 调整title的位置
colorbar.ax.tick_params(labelsize=8)
colorbar.outline.set_linewidth(.5)
colorbar.ax.tick_params(width=.5)

plt.show()